In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.signal import butter, hilbert, filtfilt, freqz
import random
import sys
sys.path.append('/Volumes/kaylab/Code.Repository/Python/')

import OpenEphys

sf = 30000

ddir = '/Volumes/kaylab/DataStores/Data/BO/SiProbe/' # data directory

# choose data folder where files are stored
dfold = 'RK90_2016-09-11_raw_30k_30pres_EMB'
#clust1 = [2,3,4,5,6,7,8] # clusters 7&8 are both CH2 spikes
#clust2 = [2,3,4,5,6,7]
#clust_names = ['sh1 - multi','sh1 - multi','sh1 - multi','sh1 - CH0','sh1 - CH8',
#              'sh1 - CH2','sh1 - CH2','sh2 - multi','sh2 - multi','sh2 - CH8',
#              'sh2 - multi','sh2 - multi','sh2 - multi']


# number of odor presentations
nop = 30
# save figures?
sflag = 0;

In [2]:
# load LFP data
filepath = ddir + dfold + '/' + dfold + '_LFP.dat'
ssLFP = np.fromfile(filepath, dtype='float') #subsampled LFP
sssf = 3000 # subsampled frequency
# NOTE!!! Downsampling not only saves space, but is also necessary to construct filters that 
# operate in the beta range

In [ ]:
# Load events, extract odor periods
eventsname = 'all_channels.events'
filepath = ddir + dfold + '/' + eventsname
Events = OpenEphys.loadEvents(filepath)
odor_periods = Events['timestamps']-Events['timestamps'][0]
odor_periods = odor_periods[2:]
# For some odd reason the TTL pulses will sometimes duplicate a few miliseconds after eachother
# Smooth over the duplicates by binning time in 0.3s bins
dl = 10*np.squeeze(ssLFP.shape) # data length
a,b = np.histogram(odor_periods,np.arange(0,dl,0.3*sf))
b = b[1:] # The first edge is 0 so we skip it
odor_periods = b[a>0] # index where bin count is > 1
# For some reason the last TTL OFF is not saved, so I add one 3s after the last ON
odor_periods = np.hstack((odor_periods,odor_periods[-1]+3*sf))

In [8]:
def load_spikes_from_kwik(dfile, clust_choice, use_clust_choice):
    '''
    Inputs : dfile        - file directory (including filename)
             clust_choice - list of cluster labels you want to extract
        
    Outputs: SPK          - list of spike times for each cluster
    '''
    import h5py
    with h5py.File(dfile,'r') as D:
        # for viewing contents of folders
        # dv=D['/channel_groups/0/spikes/time_samples']
        # for i in iter(dv):
            # print(i)
        time_samples = D.get('/channel_groups/0/spikes/time_samples')
        np_time_samples = np.array(time_samples)
        cluster_labels = D.get('/channel_groups/0/spikes/clusters/main')
        np_cluster_labels = np.array(cluster_labels)
        
    SPK = []
    if use_clust_choice == 1:
        for i in range(len(clust_choice)):
            temp_ind = np.squeeze(np.array(np.where(np_cluster_labels == clust_choice[i])))
            SPK.append(np_time_samples[temp_ind])
    else:
        for i in range(len(np_cluster_labels)):
            temp_ind = np.squeeze(np.array(np.where(np_cluster_labels == np_cluster_labels[i])))
            SPK.append(np_time_samples[temp_ind])
    return SPK


# load spike times of each cluster for both shanks
dfile1 = ddir + dfold + '/2017/shank1CMR/' + dfold + '_shank1CMR' + '.kwik'
SPK1 = load_spikes_from_kwik(dfile1, [], 0)
    
dfile2 = ddir + dfold + '/2017/shank2CMR/' + dfold + '_shank2CMR' + '.kwik'
SPK2 = load_spikes_from_kwik(dfile2, [], 0)

#SPK = SPK1+SPK2
# Spike times from both are saved in a single list SPK!

In [ ]:
# Get cluster labels
import h5py
with h5py.File(dfile1,'r') as D:
    dv=D['/channel_groups/0/clusters/original']
    for i in iter(dv):
        print(i)


In [ ]:
# SpikeStats


# Plot ISI distributions
sflag = 1

# shank1
nbins = 50
plt.figure(figsize=(28,2))
for i in range(len(clust1)):
    ISI_temp = 1000*(SPK[i][1:]-SPK[i][0:-1])/sf
    plt.subplot(1,len(clust1),i+1)
    ISI_temp = ISI_temp[ISI_temp != 0] # remove 0s (there shouldn't be any)
    plt.hist(ISI_temp, bins=np.logspace(-1,4,nbins), edgecolor = 'none')
    plt.gca().set_xscale('log')
    plt.title(clust_names[i]+'\n'+str(len(SPK[i]))+' spikes')
    plt.xlabel('ms')
    if i == 0:
        plt.ylabel('Count')
if sflag == 1:
    plt.savefig('figs/'+dfold+'_ISI1.png', bbox_inches='tight')
        
# shank2
nbins = 50
plt.figure(figsize=(20,2))
for i in range(len(clust2)):
    ISI_temp = 1000*(SPK[i+len(clust1)][1:]-SPK[i+len(clust1)][0:-1])/sf
    plt.subplot(1,len(clust2),i+1)
    ISI_temp = ISI_temp[ISI_temp != 0] # remove 0s (there shouldn't be any)
    plt.hist(ISI_temp, bins=np.logspace(-1,4,nbins), edgecolor = 'none')
    plt.gca().set_xscale('log')
    plt.title(clust_names[i+len(clust1)]+'\n'+str(len(SPK[i+len(clust1)]))+' spikes')
    plt.xlabel('ms')
    if i == 0:
        plt.ylabel('Count')
if sflag == 1:
    plt.savefig('figs/'+dfold+'_ISI2.png', bbox_inches='tight')    
# log(x) = 10 for x~20000 (20s)

In [ ]:
# Extract LFP in odor periods
start = odor_periods[0::2] - sf # add 1s to each end of odor period
stop = odor_periods[1::2] + sf

ssLFP_op = list()
SPK_op = list()
op_num_spk = np.zeros(len(SPK))

for i in range(nop):
    # remember, LFP is subsampled by 10
    ssLFP_op.append(ssLFP[start[i]/10:stop[i]/10])
    SPKtmp = list()
    op_num_spk_temp = np.zeros(len(SPK))
    for s in range(len(SPK)):
        # find spk times in odor periods. Don't save spikes within 10 smaples of the stop
        # to avoid later indexing issues when indexing last element of subsampled array
        sind = np.array(np.where((SPK[s] > start[i]) & (SPK[s] < (stop[i]-10))))
        sind = sind.astype(int) # convert type to int
        SPKtmp.append(SPK[s][sind]-start[i]) # subtract away start time of odor period
        op_num_spk_temp[s] = sind.size
    SPK_op.append(SPKtmp)
    op_num_spk = op_num_spk+op_num_spk_temp # total # of spikes in odor periods
    


In [ ]:
# Plot odor periods, LFP, and spikes for entire session
includeHP = 0
sflag = 0
sstime = np.linspace(0,ssLFP.size/sssf,ssLFP.size) # subsampled time

x1 = 0
x2 = 20
plt.figure(figsize=(16,10))
plt.eventplot(odor_periods/sf, orientation='horizontal', lineoffsets=300, linelengths=200, 
              linewidths=2, colors='k', linestyles='solid', hold=None, data=None)
plt.text(x2+0.01*(x2-x1),300,'odor periods')
plt.plot(sstime,0.1*ssLFP,color = 'b')
plt.text(x2+0.01*(x2-x1),0,'LFP',color = 'b')
if includeHP == 1:
    plt.plot(np.linspace(0,CONT1.size/sf,CONT1.size),0.3*CONT1-250,color = 'k')
    plt.text(x2+0.01*(x2-x1),-250,'sh1 - highpass',color = 'k')
    plt.plot(np.linspace(0,CONT2.size/sf,CONT2.size),0.3*CONT2-350,color = 'k')
    plt.text(x2+0.01*(x2-x1),-350,'sh2 - highpass',color = 'k')
for i in range(len(SPK)-1):
    plt.eventplot(SPK[i]/sf, orientation='horizontal', lineoffsets=-500-i*50, linelengths=40, 
              linewidths=1, colors='k', linestyles='solid', hold=None, data=None,label = 'TTL')
    plt.text(x2+0.01*(x2-x1),-500-i*50,clust_names[i])
plt.xlim([x1,x2]),plt.xlabel('Time (s)')
if sflag == 1:
    plt.savefig('figs/'+dfold+'_OP-LFP-HP-SPK_1st20s.png', bbox_inches='tight')
    # plt.savefig('figs/'+dfold+'_beta-epochs3.png', bbox_inches='tight')
    # plt.savefig('figs/'+dfold+'_PeriodicBurst2_zoom.png', bbox_inches='tight')
    # plt.savefig('figs/'+dfold+'_GammaLeak?.png', bbox_inches='tight')


In [14]:
import h5py
with h5py.File(dfile1,'r') as D:
    # for viewing contents of folders
    # dv=D['/channel_groups/0/spikes/time_samples']
    # for i in iter(dv):
        # print(i)
    time_samples = D.get('/channel_groups/0/spikes/time_samples')
    np_time_samples = np.array(time_samples)
    cluster_labels = D.get('/channel_groups/0/spikes/clusters/main')
    np_cluster_labels = np.array(cluster_labels)

In [29]:
with h5py.File(dfile1,'r') as D:
    dv=D['/channel_groups/0/channels']
    for i in iter(dv):
        print(i)

0
1
10
11
12
13
14
15
2
3
4
5
6
7
8
9
